In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
from package.databases.initialize import initialize_memories
initialize_memories()

In [4]:
from package.utils.data_loder import PDFLoader
from package.interface import SourceOptions
from package.flows.offline import OfflineFlow
from package.databases.management.longterm import LongTermManagement
from package.databases.management.user import UserManagement
from package.databases.management.document import DocumentManagement
from package.databases.management.project import ProjectManagement
from package.databases.management.jargon import JargonManagement
from package.databases.session import Session, get_session, Depends
from package.databases.models.user import User
from package.databases.models.document import Document
from package.databases.models.project import Project
from package.databases.models.jargon import Jargon
from package.databases.models.longterm import LongTerm

d:\broai-arai\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\broai-arai\backend\package\databases\management\longterm.py:9: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 29987.87it/s]


In [8]:
user = User(
    username="bank",
    password="555",
    email="bank@bank.com"
)

um = UserManagement()
user = um.create_user(user, session=Depends(get_session))

In [18]:
user.id

'de8f4577-7800-4be5-a4e8-86f7df3ad2aa'

In [9]:
document1 = Document(source="./sources/storm.pdf", type="pdf")

dm = DocumentManagement()
document1 = dm.create_document(document1, session=Depends(get_session)) 

In [10]:
document1.id

'3ec94e05-d642-433c-b4c5-74740b3f636d'

In [21]:
dm.read_document_longterms(document_id=document1.id, session=Depends(get_session))

[LongTerm(id='bb41b6f8-04ea-4025-a6d9-91171260835c', enrich=None, combo=None, meta={'type': 'pdf', 'source': './sources/storm.pdf', 'section': '1 Introduction', 'sequence': 4}, created_at=datetime.datetime(2025, 6, 28, 16, 18, 23, 785699), raw="outlines or even entire articles ( Direct Gen ). However, this approach is limited by a lack of details and hallucinations (Xu et al., 2023), particularly in addressing long-tail topics (Kandpal et al., 2023). This underscores the importance of leveraging external sources, and current strategies often involve retrieval-augmented generation ( RAG ), which circles back to the problem of researching the topic in the pre-writing stage, as much information cannot be surfaced through simple topic searches.\n\nHuman learning theories (Tawfik et al., 2020; Booth et al., 2003) highlight asking effective questions in information acquisition. Although instruction-tuned models (Ouyang et al., 2022) can be prompted directly to generate questions, we find tha

In [11]:
source_ops = SourceOptions(path="./sources/storm.pdf", type="pdf")
loader = PDFLoader(
    source=source_ops
)
contexts = loader.run()

Markdown headings: max(2)


d:\broai-arai\backend\package\utils\data_loder.py:22: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(text)
d:\broai-arai\backend\package\utils\data_loder.py:23: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)
d:\broai-arai\backend\package\utils\data_loder.py:24: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)
d:\broai-arai\backend\package\utils\data_loder.py:30: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts, max_tokens=max_tokens, overlap=overlap)


In [12]:
olf = OfflineFlow()

In [13]:
longterms = olf.run(document_id=document1.id, contexts=contexts)

In [14]:
ltm = LongTermManagement()
ltm.create_raws(longterms, session=Depends(get_session))

In [22]:
ltm.embed_texts(document_id=document1.id, embed_method="raw", session=Depends(get_session))

In [25]:
query = """What does ROUGE Score do?"""
results = ltm.read_similar_text(query, limit=15, embed_method="raw", sources=[document1.source], session=Depends(get_session))

In [26]:
for result in results:
    print(result.meta)

{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '4.4 STORM Implementation', 'sequence': 20}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '6 Human Evaluation', 'sequence': 27}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'C.3 More Discussion of the Citation Quality', 'sequence': 52}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'E Error Analysis', 'sequence': 60}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'C.3 More Discussion of the Citation Quality', 'sequence': 49}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '6 Human Evaluation', 'sequence': 26}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '4.2 Automatic Metrics', 'sequence': 15}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '4.4 STORM Implementation', 'sequence': 19}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'C.3 More Discussion of the Citation Quality', 'sequence': 51}
{'type': 'pdf', 'source': './s

In [6]:
from package.databases.destroy import drop_all_tables

drop_all_tables()

✅ All tables dropped.
